In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import plotly.express as px
from pickle import dump
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

/home/ben/.local/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-08 16:25:25.585918: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 16:25:25.634334: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 16:25:25.635191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 16:25:26.404944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def preprocess_data(df: pd.DataFrame)-> pd.DataFrame:
    """
    Takes a CSV from querying DynamoDB and preprocesses the data to the 
    required format for input to training. 

    Args: 
        df: a dataframe of readings, including temperature and humidity.
    
    Returns: 
        df: a dataframe of temperature readings, sorted by timestamp
    """
    df.rename(columns={'humidity.S': 'humidity',
                   'temperature.S':'temperature',
                   'timestamp.S':'timestamp'},inplace=True)
    
    # Convert the timestamp column to datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')

    # Round the timestamp to the nearest minute
    df['timestamp'] = df['timestamp'].dt.round('1min')

    # Remove unnecessary columns
    df.drop(columns=['Unnamed: 0', 'humidity'],inplace=True)
    df.sort_values(by='timestamp')
    df = df[ df['timestamp'] > '2023-04-28' ]
    df.set_index('timestamp', inplace=True)
    df.dropna(inplace=True)

    return pd.DataFrame(df)

In [23]:
def augment_missing_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Augments the data by looking for gaps of over 15 minutes, then inserting past
    temperatures from 24 hours ago. 

    Args:
        df: a dataframe of temperature readings, sorted by timestamp

    Returns:
        df: a dataframe of temperature readings, sorted by timestamp, augmented
        to input missing data with the value from 24 hours previous. 
    """

    print(f'Dataframe size before augmentation: {df.shape}')

    df_original_shape = df.shape
    one_day = 10 * 24
    time_interval = timedelta(minutes=10)
    i = one_day 

    while i < df.shape[0]-1: 

        current_time = pd.Timestamp(df.index[i])
        next_time = pd.Timestamp(df.index[i + 1])
        
        if (next_time - current_time) > time_interval + timedelta(minutes=60):
            
            previous_value_temp = df.iloc[i+1-one_day]['temperature']    
            new_row = pd.DataFrame({'temperature': previous_value_temp}, index=[pd.Timestamp(current_time + time_interval)])      
            df = pd.concat([df.iloc[:i+1], new_row, df.iloc[i+1:]])

        i += 1

    print(f'Dataframe size after empty rows added: {df.shape}')
    print(f'Rows added, {df.shape[0]-df_original_shape[0]},' 
          f'or {np.round((df.shape[0]-df_original_shape[0])*100/df.shape[0],2)}% of the new total.')

    return df


In [4]:
def train_val_test_split(df: pd.DataFrame)-> pd.DataFrame:
    """
    Takes a preprocessed dataframe, assumed to be sorted chronologically,
    and returns three dataframes split into train, validation, and test; 
    done chronologically in a 60%/20%/20% split. 

    Args: 
        df: a DataFrame

    Returns:
        df_train, df_test, df_val: a tuple of the train, test, and validation 
        data.
    """

    train_index = int(np.round(df.shape[0]*0.6))
    val_index = int(np.round(df.shape[0]*0.8))

    df_train = df.iloc[:train_index,]
    df_val = df.iloc[train_index:val_index]
    df_test = df.iloc[val_index:]

    return df_train, df_test, df_val

In [5]:
def scale_data(df_train, df_val, df_test):
    """Scales data and saves scaling object as a pickle file. 
    
    """

    scaler = StandardScaler()
    print(scaler.fit(df_train))
    print(scaler.mean_)
    print(scaler.scale_)

    dump(scaler, open('scaler.pkl', 'wb'))

    return scaler.transform(df_train), scaler.transform(df_val), scaler.transform(df_test)

In [6]:
def generate_sequences(df_train, df_val, df_test):
    """
    Creating the generators. The task will be to take in one hour of readings, 
    spaced 10 minutes apart, and predict the temperature in two hours. 
    
    For example, there will be readings at 3:00pm, 3:10pm, ..., 4:00pm, 
    and the task will be to predict the temperature at 6pm. 

    """

    delay = 24
    sequence_length = 12

    train = keras.preprocessing.timeseries_dataset_from_array(df_train[:-delay], 
                df_train[sequence_length+delay:], sequence_length=sequence_length, 
                batch_size=1, shuffle=True)

    validation = keras.preprocessing.timeseries_dataset_from_array(df_val[:-delay], 
                df_val[sequence_length+delay:], sequence_length=sequence_length, 
                batch_size=1, shuffle=True)

    test = keras.preprocessing.timeseries_dataset_from_array(df_test[:-delay], 
                df_test[sequence_length+delay:], sequence_length=sequence_length, 
                batch_size=1, shuffle=True)
    
    return train, validation, test

In [28]:
def run_preprocessing_pipeline(csv_input_path: str)-> list:
    """
    Run pipeline from CSV to generators ready for input to training. 

    Args:
        csv_input_path: a string of the path to the location of the CSV

    Returns: 
        A tuple (df_train, df_val, df_test), containing three generators 
        which load 12 inputs and 1 target, from a scaled dataset of temperature 
        readings. 
    """

    df_raw = pd.read_csv(csv_input_path)

    df_preprocessed = preprocess_data(df_raw)
    df_augmented = augment_missing_data(df_preprocessed)
    df_train, df_val, df_test = train_val_test_split(df_augmented)
    df_train, df_val, df_test = scale_data(df_train, df_val, df_test)

    return generate_sequences(df_train, df_val, df_test)

train, validation, test = run_preprocessing_pipeline('../ddb_output.csv')

Dataframe size before augmentation: (17104, 1)
Dataframe size after empty rows added: (18396, 1)
Rows added, 1292,or 7.02% of the new total.
StandardScaler()
[23.12753941]
[2.05427216]


In [ ]:
# To run when loading the scaler object back

# from pickle import loada
# scaler_loaded = load(open('scaler.pkl', 'rb'))
# s_test = scaler_loaded.transform(df_test)
# np.array_equal(s_test, df_test_scaled)

In [12]:
df_raw = pd.read_csv('../ddb_output.csv')
print(df_raw.shape)
df_preprocessed = preprocess_data(df_raw)
print(df_preprocessed.shape)


(17104, 4)
(17104, 1)


In [13]:
import plotly.express as px

In [14]:
df_preprocessed

,temperature
timestamp,
2023-04-28 08:25:00,17.83
2023-04-28 08:26:00,17.85
2023-04-28 08:28:00,17.85
2023-04-28 08:32:00,17.88
2023-04-28 09:30:00,17.92
...,...
2023-12-08 15:40:00,18.09
2023-12-08 15:50:00,18.19
2023-12-08 16:00:00,18.16


In [17]:
fig = px.scatter(df_preprocessed, y="temperature", title='All-time humidity and temperature in the grove!')
fig.show()

In [24]:
df_augmented = augment_missing_data(df_preprocessed)

Dataframe size before augmentation: (17104, 1)
Dataframe size after empty rows added: (18396, 1)
Rows added, 1292,or 7.02% of the new total.


In [27]:
fig = px.scatter(df_augmented, y="temperature", title='All-time humidity and temperature in the grove!')
fig.show()